APPENDIX 2: ADVANCED DQN: DOUBLE AND Deuling

Base code from https://github.com/Curt-Park/rainbow-is-all-you-need with adaptations from https://github.com/philtabor/Youtube-Code-Repository/tree/master/ReinforcementLearning/DeepQLearning, and https://www.youtube.com/watch?v=WHRQUZrxxGw&t=4786s

In [ ]:
from torch.cuda import random
!pip install Box2D
import os
from typing import Dict, List, Tuple

import gym
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from IPython.display import clear_output
from torch.nn.utils import clip_grad_norm_

!pip install box2d-py
!pip install gym[all]
!pip install gym[Box_2D]
!pip install git+https://github.com/ngc92/space-wrappers.git
!pip install "ray[rllib]"==1.6

import gym
import space_wrappers

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import randint

from collections import deque,namedtuple
from typing import Any
from random import sample, random

from dataclasses import dataclass
from ray import tune
import numpy as np
import ray.rllib.agents.dqn as dqn

In [ ]:
!pip install wandb -qqq
import wandb

In [ ]:
wandb.login()

In [ ]:
from random import sample, random


In [ ]:
import torch
import torch.nn as nn

import torch.optim as optim
import torch.nn.functional as F

In [ ]:
env = gym.make('LunarLander-v3').env




Create the Q network - with target

In [ ]:
class DQN(nn.Module):
    def __init__(self, inputs_states, output_actions):
        super(DQN, self).__init__()
        self.inputs_states = inputs_states
        self.output_actions = output_actions
        self.fc1 = nn.Linear(inputs_states, 256)
        self.fc2 = nn.Linear(256, 256)
        #self.fc3 = nn.BatchNorm1d(hidden, hidden)
        self.fc4 = nn.Linear(256, output_actions)

        self.optimizer = optim.Adam(self.params(), lr=0.01)


    def forward(self, x):
        x = F.relu(self.fc1(x))
        #x = self.fc3(x)
        x = F.relu(self.fc2(x))
        #x = self.fc3(x)
        x = self.fc4(x)

        return x
    
    def training_step(self, state_transitions):
      current_states = torch.stack([s.state for s in state_transitions])
      rewards = torch.stack([s.reward for r in state_transitions])
      next_states = torch.stack([s.new_state for ns in state_transitions])
      if_done = torch.stack([0 if s.done else 1 for s in state_transitions])
      actions = torch.stack([s.action for a in state_transitions])

      




In [ ]:
env1 = gym.make('BipedalWalker-v3')
wrapped = space_wrappers.DiscretizedActionWrapper(env1, 3)
wrapped = space_wrappers.FlattenedActionWrapper(wrapped)
env = wrapped

#Initialize Network


#gets trained every step
q_evaluation = DQN(env)
#gives predictions every step
q_target = DQN(env)

q_target.load_state_dict(q_evaluation.state_dict)


#initialize buffer
obs = env.reset()

#get observations and tuple






Create action replay (taken from https://towardsdatascience.com/deep-q-network-with-pytorch-and-gym-to-solve-acrobot-game-d677836bda9b)

In [ ]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

In [ ]:
from typing import Any
@dataclass
class SARNS():
  state: Any
  action: Any
  reward: Any
  next_state : Any


class replay(object):
  def __init__(self, memory_size):
    self.memory = deque([], maxlen = memory_size)
    self.capacity = memory_size

  def insert(self, SARNS):
    #save a tranition
    self.memory.append(SARNS)

  def samples(self, batch_size): #also used in pritorized replay
    batch_size = min(batch_size, len(self))
    return sample(self.memory, batch_size)
  
  def __len__(self):
    return len(self.memory)

In [ ]:
p = replay(1000)

In [ ]:
def training_step(model, state_transitions, tgt, num_actions):
      #Get transitions
      current_states = torch.stack([torch.tensor(s.state) for s in state_transitions])
      rewards = torch.stack([torch.tensor(s.reward) for s in state_transitions])
      next_states = torch.stack([torch.tensor(s.next_state) for s in state_transitions])
      if_done = torch.stack(
          [torch.tensor([0]) if s.done else torch.tensor([1]) for s in state_transitions]
                          )
      actions = torch.stack([torch.tensor(s.action) for s in state_transitions])


      with torch.no_grad():
        next_qvals = q_target().max(-1)

      model.opt.zero_grad()
      quals = model(current_states)
      one_hot_actions = F.one_hot(torch.LongTensor(actions), num_actions)

      rewards + q_evaluation()

In [ ]:
class Agent():
  def __init__(self, lr, gamma, n_actions, batch_size, input_dims, epsilon, epsilon_decay = 1e-3, epsilon_min=0.01, memory_size=1000000):
    self.action = [action for action in range(n_actions)]
    self.gamma = gamma
    self.batch_size = batch_size
    self.epsilon = epsilon
    self.epsilon_min = epsilon_min
    self.memory = replay(memory_size)
    self.q_eval = DQN...
    self.q_tar = DQN
    self.device = 'cuda' if torch.cuda.is_available() else 'cpu'


  def store_transition():
    pass

  def choose_action(self, obs):
    if np.random.random() < self.epsilon:
      action = np.random.choice(self.action.space) #...checkthis
    else:
      state = 
      actions = 
      action = np.argmax(actions)

    return action

  def learn(self):
    #if memory is less than batch size than pass
    #else add to batch



In [ ]:
#copied varbatim

import random

class E_Greedy_Policy():
    
    def __init__(self, epsilon, decay, min_epsilon):
        
        self.epsilon = epsilon
        self.epsilon_start = epsilon
        self.decay = decay
        self.epsilon_min = min_epsilon
                
    def __call__(self, state):
                
        is_greedy = random.random() > self.epsilon
        
        if is_greedy :
            # we select greedy action
            with torch.no_grad():
                Q_network.eval()
                # index of the maximum over dimension 1.
                index_action = Q_network(state).max(1)[1].view(1, 1).cpu()[0][0].item()
                
                Q_network.train()
        else:
            # we sample a random action
            index_action = env.action_space.sample()
        
        return index_action
                
    def update_epsilon(self):
        
        self.epsilon = self.epsilon*self.decay
        if self.epsilon < self.epsilon_min:
            self.epsilon = self.epsilon_min
        
    def reset(self):
        self.epsilon = self.epsilon_start
        
        
policy = E_Greedy_Policy(0.99, decay=0.997, min_epsilon=0.001)

In [ ]:
def convert_state(state):
    state_tensor = torch.tensor(state_tensor, device=device)
    return state_tensor

In [ ]:
memory = replay(10000)


In [ ]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

# Get screen size so that we can initialize layers correctly based on shape
# returned from AI gym. Typical dimensions at this point are close to 3x40x90
# which is the result of a clamped and down-scaled render buffer in get_screen()
#init_screen = get_screen()
#_, _, screen_height, screen_width = init_screen.shape

# Get number of actions from gym action space
n_actions = env.action_space.n

policy_net = DQN(24, 48).to(device)
target_net = DQN(24, 48).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.SGD(policy_net.parameters(), 0.01)
memory = replay(10000)


steps_done = 0

import random
import math

def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)

In [ ]:
def model_loop():
    if len(memory) < BATCH_SIZE:
      return
  
    transitions = memory.sample(BATCH_SIZE)

    batch = Transition(*zip(*transitions))


    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)


    state_action_values = policy_net(state_batch).gather(1, action_batch)


    next_state_values = torch.zeros(BATCH_SIZE, device=device)

    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()

    expected_state_action_values = (next_state_values * GAMMA) + reward_batch


    criterion = nn.mse_loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
      param.grad.data.clamp_(-1, 1)
    optimizer.step()


In [ ]:

class choose_epsgre_action():

  def __init_(self, epsilon, decay, min_epsilon):
    self.epsilon = epsilon
    self.decay = decay
    self.epsilon_min = min_epsilon

  def __call__(self, state):
      if random.random() > self.epsilon:
        with torch.no_grad():
          Q_network.eval()

          index_action = Q_network(state).max(1)[1].view(1, 1).cpu()[0][0].item()

      else:
            # we sample a random action
            index_action = random.randint(0,3)
        
      return index_action

  def update_epsilon(self):
        
        self.epsilon = self.epsilon*self.decay
        if self.epsilon < self.epsilon_min:
            self.epsilon = self.epsilon_min
        
  def reset(self):
        self.epsilon = self.epsilon_start
        
        
policy = choose_epsgre_action(0.99, decay=0.997, min_epsilon=0.001)




In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

OBS_SIZE = 24
HIDDEN_SIZE = 64
ACTION_SIZE = 81

Q_network = DQN(OBS_SIZE, HIDDEN_SIZE).to(device)
Q_target = DQN(OBS_SIZE, HIDDEN_SIZE).to(device)
Q_target.load_state_dict(Q_network.state_dict())
Q_target.eval()

TARGET_UPDATE = 20

optimizer = optim.SGD(Q_network.parameters(), lr=0.01)
memory = replay(10000)

In [ ]:
from collections import namedtuple

Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

In [ ]:
BATCH_SIZE = 256
GAMMA = 0.5

def optimize_model():
    
    transitions = memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
    
    # Compute Q values using policy net
    Q_values = Q_network(state_batch).gather(1, action_batch)

    # Compute next Q values using Q_targets
    next_Q_values = torch.zeros( BATCH_SIZE, device=device)
    next_Q_values[non_final_mask] = Q_target(non_final_next_states).max(1)[0].detach()
    next_Q_values = next_Q_values.unsqueeze(1)
    
    # Compute targets
    target_Q_values = (next_Q_values * GAMMA) + reward_batch
    
    # Compute MSE Loss
    loss = F.mse_loss(Q_values, target_Q_values)
    
    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    
    # Trick: gradient clipping
    for param in Q_network.parameters():
        param.grad.data.clamp_(-1, 1)
        
    optimizer.step()
    
    return loss

In [ ]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [ ]:
num_episodes = 50
for i_episode in range(num_episodes):
    # Initialize the environment and state
    env.reset()
    #last_screen = get_screen()
    #current_screen = get_screen()
    state = env.reset()
    for t in count():
        # Select and perform an action
        action = select_action(state)
        _, reward, done, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)

        # Observe new state
        last_screen = current_screen
        current_screen = get_screen()
        if not done:
            next_state = current_screen - last_screen
        else:
            next_state = None

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()
        if done:
            episode_durations.append(t + 1)
            plot_durations()
            break
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')
env.render()
env.close()
plt.ioff()
plt.show()

DQN LOOP

In [ ]:
for i in range(episodes):
  score = 0
  done = False
  state = env.reset()

  while not done:
    #choose action
    action = agent.choose_action(state) #choose action epsilon greedy action

    #commit action
    state_, reward, done, info = env.step(action)
    score += reward

    #store transitions
    agent.store_transition(state, action, reward, done, state_)

    agent.learn()
    state = state_



MAIN LOOP

TEsting

In [ ]:
def training_step(state_transitions):
      #Get transitions
      current_states = torch.stack([torch.tensor(s.state) for s in state_transitions])
      rewards = torch.stack([torch.tensor(s.reward) for s in state_transitions])
      next_states = torch.stack([torch.tensor(s.next_state) for s in state_transitions])
      #if_done = torch.stack([torch.tensor([0]) if s.done else torch.tensor([1]) for s in state_transitions])
      actions = torch.stack([torch.tensor(s.action) for s in state_transitions])

      #with torch.no_grad():
        #next_qvals = q_target().max(-1)

      #rewards + q_evaluation()

      return current_states, rewards, next_states, actions

In [ ]:
def training_step(model, state_transitions, tgt, num_actions): #target model
      #Get transitions
      current_states = torch.stack([torch.tensor(s.state, dtype=torch.float32) for s in state_transitions])
      rewards = torch.stack([torch.tensor(s.reward) for s in state_transitions])
      next_states = torch.stack([torch.tensor(s.next_state, dtype=torch.float32) for s in state_transitions])
      mask = torch.stack([torch.tensor([0]) if s.done else torch.tensor([1]) for s in state_transitions])
      actions = [torch.tensor(s.action) for s in state_transitions]

      #Get Q values on next states from the target network
      with torch.no_grad():
        qvals_nextstate = tgt(next_states).max(-1)[0] #(N, num_actions)

      model.optim.zero_grad()
      #Get Q values on current states
      qvals = model(current_states)


      one_hot_actions = F.one_hot(torch.LongTensor(actions), num_actions)
      target_Q_values = (qvals_nextstate * GAMMA) + rewards
      loss = ((rewards + mask[:,0] * qvals_nextstate - torch.sum(qvals * one_hot_actions, -1))**2 ).mean()

      #compute loss
      #loss = F.mse_loss(qvals, target_Q_values)
      loss.backward()

       #Trick: gradient clipping
      for param in model.parameters():
        param.grad.data.clamp_(-1, 1)

      model.optim.step()
      return loss



In [ ]:
from typing import Any
@dataclass
class SARNS():
  state: Any
  action: Any
  reward: Any
  next_state : Any
  done: Any


class replay(object):
  def __init__(self, memory_size=10000):
    self.memory = deque([], maxlen = memory_size)
    self.capacity = memory_size

  def insert(self, SARNS):
    #save a tranition
    self.memory.append(SARNS)

  def samples(self, batch_size): #also used in pritorized replay
    batch_size = min(batch_size, len(self))
    return sample(self.memory, batch_size)
  
  def __len__(self):
    return len(self.memory)

In [ ]:
def update_tgt_model(m, tgt):
    tgt.load_state_dict(m.state_dict())



In [ ]:
env.observation_space.shape

(24,)

In [ ]:
class DQNAgent():
    def __init__(self, model):
      self.model = model
    
    def get_actions(self, observations):
      return q_vals.max(-1)[1]

In [ ]:
wandb.init(project="dqn-no-improvement", name="bot11")
GAMMA = 0.7
EPS_DECAY = 0.999999
EPS_MIN = 0.01
min_rb_size = 10000 #buffer size
sample_size = 2500 
env_step_before_train = 100
tgt_model_update = 50
eps = 0.9999

last_observation = env.reset() #get observations
#last_observation1 = torch.tensor(last_observation, dtype=torch.float32)
##m = DQN(24, 81) # evaluation model
#tgt = DQN(24, 81) #target model
#m = DQN(env.observation_space.shape[0], 81)
#tgt = DQN(env.observation_space.shape[0], 81)
update_tgt_model(m, tgt)

rb = replay(15000) #initialize replay buffer
steps_since_train = 0
epochs_since_tgt = 0

step_num = -1 * min_rb_size

eps = 0.9999
episode_rewards = []
rolling_reward = 0

#model loop
while True:
    
    eps = EPS_DECAY ** (step_num)
    
    if random() < eps:
      action = env.action_space.sample()
    else:
      action = m(torch.Tensor(last_observation)).max(-1)[-1].item()
      #action = m(torch.Tensor(last_observation1).item()#your agent here this takes random samples
    observation, reward, done, info = env.step(action)

    rolling_reward += reward
    reward = reward * 0.1

    rb.insert(SARNS(last_observation, action, reward, observation, done))

    last_observation = observation

    if done:
      episode_rewards.append(rolling_reward)
      rolling_reward = 0
      observation = env.reset()


    steps_since_train +=1
    step_num += 1


    if len(rb) > min_rb_size and steps_since_train > env_step_before_train:
      loss = training_step(m, rb.samples(sample_size), tgt, 81)
      wandb.log({'loss': loss.detach().item(), 'eps':eps, 'avg_reward': np.mean(episode_rewards)}, step = step_num)
      print(step_num, loss.detach().item(), np.mean(episode_rewards))
      episode_rewards = []
      epochs_since_tgt +=1
      if epochs_since_tgt > tgt_model_update:
        update_tgt_model(m, tgt)
        print('updating model', epochs_since_tgt)
        epochs_since_tgt = 0
      steps_since_train = 0
      print(loss)


In [ ]:
#Double DQN

In [ ]:


class ReplayBuffer:

    def __init__(self, obs_dim, size, batch_size):
        self.observations_buffer = np.zeros([size, obs_dim])
        self.next_obs_buf = np.zeros([size, obs_dim])
        self.actions_buffer = np.zeros([size])
        self.rewards_buffer = np.zeros([size])
        self.done_buffer = np.zeros(size)
        self.max_size, self.batch_size = size, batch_size
        self.ptr, self.size, = 0, 0

    def store(self, obs, act, rew, next_obs, done):
        self.observations_buffer[self.ptr] = obs
        self.next_obs_buf[self.ptr] = next_obs
        self.actions_buffer[self.ptr] = act
        self.rewards_buffer[self.ptr] = rew
        self.done_buffer[self.ptr] = done
        self.ptr = (self.ptr + 1) % self.max_size
        self.size = min(self.size + 1, self.max_size)

    def sample_batch(self):
        idx = np.random.choice(self.size, size=self.batch_size, replace=False)
        return dict(observations=self.observations_buffer[idx],
                    next_observations=self.next_obs_buf[idx],
                    actions=self.actions_buffer[idx],
                    rewards=self.rewards_buffer[idx],
                    done=self.done_buffer[idx])

    def __len__(self):
        return self.size

In [ ]:
class Network(nn.Module):
    def __init__(self, in_dim, out_dim):
        """Initialization."""
        super(Network, self).__init__()

        self.layers = nn.Sequential(
            nn.Linear(in_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, out_dim)
        )

    def forward(self, x):
        """Forward method implementation."""
        return self.layers(x)


class DQNAgent:
    def __init__(self,env: gym.Env,memory_size: int,batch_size: int,target_update: int,epsilon_decay: float,max_epsilon: float = 1.0,min_epsilon: float = 0.1,gamma: float = 0.99,):
        
        obs_dim = env.observation_space.shape[0]
        action_dim = env.action_space.n

        self.env = env
        self.memory = ReplayBuffer(obs_dim, memory_size, batch_size)
        self.batch_size = batch_size
        self.epsilon = max_epsilon
        self.epsilon_decay = epsilon_decay
        self.max_epsilon = max_epsilon
        self.min_epsilon = min_epsilon
        self.target_update = target_update
        self.gamma = gamma

        # device: cpu / gpu
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )

        # networks: dqn, dqn_target
        self.dqn = Network(obs_dim, action_dim).to(self.device)
        self.dqn_target = Network(obs_dim, action_dim).to(self.device)
        self.dqn_target.load_state_dict(self.dqn.state_dict())
        self.dqn_target.eval()

        # optimizer
        self.optimizer = optim.Adam(self.dqn.parameters())

        # transition to store in memory
        self.transition = list()


    def select_action(self, state: np.ndarray) ->:
 
        # epsilon greedy policy
        if self.epsilon > np.random.random():
            selected_action = self.env.action_space.sample()
        else:
            selected_action = self.dqn(
                torch.FloatTensor(state).to(self.device)
            ).argmax()
            selected_action = selected_action.detach().cpu().numpy()

        self.transition = [state, selected_action]

        return selected_action

    def step(self, action: np.ndarray):
        #action and next state and reward
        next_state, reward, done, _ = self.env.step(action)


        self.transition += [reward, next_state, done]
        self.memory.store(*self.transition)

        return next_state, reward, done

    def update_model(self) :
        samples = self.memory.sample_batch()

        loss = self._compute_dqn_loss(samples)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        return loss.item()

    def train(self, num_frames: int, plotting_interval: int = 200):

        state = self.env.reset()
        update_cnt = 0
        epsilons = []
        losses = []
        scores = []
        score = 0

        for frame_idx in range(1, num_frames + 1):
            action = self.select_action(state)
            next_state, reward, done = self.step(action)

            state = next_state
            score += reward

            # if episode ends
            if done:
                state = self.env.reset()
                scores.append(score)
                score = 0

            # if training is ready
            if len(self.memory):
                loss = self.update_model()
                losses.append(loss)
                update_cnt += 1

                # epsilon decay
                self.epsilon = max(
                    self.min_epsilon, self.epsilon - (
                            self.max_epsilon - self.min_epsilon
                    ) * self.epsilon_decay
                )
                epsilons.append(self.epsilon)

                # if hard update is needed
                if update_cnt % self.target_update == 0:
                    self._target_hard_update()




    def _compute_dqn_loss(self, samples: Dict[str, np.ndarray]) -> torch.Tensor:
        """Return dqn loss."""
        device = self.device  # for shortening the following lines
        state = torch.FloatTensor(samples["obs"]).to(device)
        next_state = torch.FloatTensor(samples["next_obs"]).to(device)
        action = torch.LongTensor(samples["acts"].reshape(-1, 1)).to(device)
        reward = torch.FloatTensor(samples["rews"].reshape(-1, 1)).to(device)
        done = torch.FloatTensor(samples["done"].reshape(-1, 1)).to(device)


        curr_q_value = self.dqn(state).gather(1, action)
        next_q_value = self.dqn_target(next_state).gather(  # Double DQN
            1, self.dqn(next_state).argmax(dim=1, keepdim=True)
        ).detach()
        mask = 1 - done
        target = (reward + self.gamma * next_q_value * mask).to(self.device)

        # calculate dqn loss
        loss = F.smooth_l1_loss(curr_q_value, target)

        return loss

    def _target_hard_update(self):
        self.dqn_target.load_state_dict(self.dqn.state_dict())


Deuling DQN

In [ ]:
class Network(nn.Module):
    def __init__(self, observations_dim, output_dim):
        super(Network, self).__init__()

        # set common feature layer
        self.common_stream = nn.Sequential(
            nn.Linear(observations_dim, 128),
            nn.ReLU())

        #advantage function for each action a =  (A(s, a))
        self.advantage_stream = nn.Sequential(
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim))

        # #state value function for state s =  (V(s))
        self.value_stream = nn.Sequential(
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 1))

    def forward(self, x):
        feature = self.common_stream(x)

        value = self.value_stream(feature)
        advantage = self.advantage_stream(feature)
        # infinitely many solutions so trick to make problem identifiable
        # we take mean
        q = value + advantage - advantage.mean(dim=-1, keepdim=True)

        return q


class DQNAgent:
 
    def __init__( self, env, memory_size, batch_size, target_update, epsilon_decay, max_epsilon = 1.0, min_epsilon = 0.1,gamma = 0.99,):

        obs_dim = env.observation_space.shape[0]
        action_dim = env.action_space.n

        self.env = env
        self.memory = ReplayBuffer(obs_dim, memory_size, batch_size)
        self.batch_size = batch_size
        self.epsilon = max_epsilon
        self.epsilon_decay = epsilon_decay
        self.max_epsilon = max_epsilon
        self.min_epsilon = min_epsilon
        self.target_update = target_update
        self.gamma = gamma

        # device: cpu / gpu
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu")

        # networks: dqn, dqn_target
        self.dqn = Network(obs_dim, action_dim).to(self.device)
        self.dqn_target = Network(obs_dim, action_dim).to(self.device)
        self.dqn_target.load_state_dict(self.dqn.state_dict())
        self.dqn_target.eval()

        # optimizer
        self.optimizer = optim.Adam(self.dqn.parameters())

        # transition to store in memory
        self.transition = list()

    def select_action(self, state):
        # epsilon greedy policy
        if self.epsilon > np.random.random():
            selected_action = self.env.action_space.sample()
        else:
            selected_action = self.dqn(
                torch.FloatTensor(state).to(self.device)
            ).argmax()
            selected_action = selected_action.detach().cpu().numpy()


        self.transition = [state, selected_action]

        return selected_action

    def step(self, action):
        next_state, reward, done, _ = self.env.step(action)

        self.transition += [reward, next_state, done]
        self.memory.store(*self.transition)

        return next_state, reward, done

    def update_model(self):
        samples = self.memory.sample_batch()

        loss = self._compute_dqn_loss(samples)

        self.optimizer.zero_grad()
        loss.backward()

        # trick: gradient clipping.
        clip_grad_norm_(self.dqn.parameters(), 10.0)
        self.optimizer.step()

        return loss.item()

    def train(self, num_steps, plotting_interval = 200):
#train agent
        state = self.env.reset()
        update_target_count = 0
        epsilons, losses, scores = [], [], []
        total_ep_reward = 0

        for steps in range(num_steps):
            action = self.select_action(state)
            next_state, reward, done = self.step(action)

            state = next_state
            total_ep_reward += reward

            # if episode ends
            if done:
                state = self.env.reset()
                scores.append(total_ep_reward)
                total_ep_reward = 0

            # if training is ready
            if len(self.memory) >= self.batch_size:
                loss = self.update_model()
                losses.append(loss)
                update_target_count += 1

                #epsilon decay
                self.epsilon = max(self.min_epsilon, self.epsilon - (self.max_epsilon - self.min_epsilon) * self.epsilon_decay)
                epsilons.append(self.epsilon)

                # if hard update is needed
                if update_target_count % self.target_update == 0:
                    self.update_target_function()


    def _compute_dqn_loss(self, samples):
        """Return dqn loss."""
        device = self.device
        state = torch.FloatTensor(samples["observations"]).to(device)
        next_state = torch.FloatTensor(samples["next_observations"]).to(device)
        action = torch.LongTensor(samples["actions"].reshape(-1, 1)).to(device)
        reward = torch.FloatTensor(samples["rewards"].reshape(-1, 1)).to(device)
        done = torch.FloatTensor(samples["done"].reshape(-1, 1)).to(device)

        curr_q_value = self.dqn(state).gather(1, action)
        next_q_value = self.dqn_target(next_state).max(dim=1, keepdim=True)[0].detach()
        mask = 1 - done
        target = (reward + self.gamma * next_q_value * mask).to(self.device)

        # calculate dqn loss
        loss = F.smooth_l1_loss(curr_q_value, target)

        return loss

    #update target network
    def update_target_function(self):
        self.dqn_target.load_state_dict(self.dqn.state_dict())


#training loop

state = env.reset()
update_target_count = 0
epsilons, losses, reward_list = [], [], []
total_ep_reward = 0

for steps in range(num_steps):
    action = select_action(state)
    next_state, reward, done = step(action)

    state = next_state
    total_ep_reward += reward

    # if episode ends
    if done:
        state = env.reset()
        reward_list.append(total_ep_reward)
        total_ep_reward = 0

    # if training is ready
    if len(memory) >= batch_size:
        loss = update_model()
        losses.append(loss)
        update_target_count += 1

        #epsilon decay
        epsilon = max(min_epsilon, epsilon - (max_epsilon - min_epsilon) * epsilon_decay)
        epsilons.append(epsilon)

        # target update
        if update_target_count % target_update == 0:
            update_target_function()